In [ ]:
# Set environment seed
import os
os.environ['PYTHONHASHSEED']=str(1)

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import datetime
import json
from contextlib import redirect_stdout
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import train_test_split
import time

from tensorflow import keras
from tensorflow.keras.utils import plot_model

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
def reset_random_seeds():
    '''
    Sets all necessary seed for reproduceability.
    '''
    os.environ['PYTHONHASHSEED']=str(1)
    tf.random.set_seed(1)
    np.random.seed(1)
    
reset_random_seeds()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('https://drive.google.com/uc?id=13skTlmf1OiiM8sN6eoLPTToJu5zfdzui&export=download')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
# Read the dataset and perofrm stratified train test split

x_train, x_test, y_train, y_test = train_test_split(data['review'], data['sentiment'], 
                                                    test_size=0.2, random_state=0, 
                                                    stratify=data['sentiment'])

In [ ]:
# Set the hyper parameters
hyper_params = {
    'VOCAB_SIZE':50000,
    'EPOCHS':150,
    'BS':512,
    'LR':0.01,
    'OOV_TOK':""
}


In [ ]:
# Tokenize on the training dataset and apply it on our training and testing dataset

tokenizer = Tokenizer(num_words = hyper_params['VOCAB_SIZE'], oov_token=hyper_params['OOV_TOK'])
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(x_train)
test_sequences = tokenizer.texts_to_sequences(x_test)

In [ ]:
# visualize the statistics for the number of words in a sentence for any sentence in our training dataset 

num_words = []
for item in (train_sequences):
    num_words.append(len(item))
fig = px.box(num_words)
fig.show()

NameError: ignored

In [ ]:
# We will set the max length to 520, the upper fence value.

hyper_params['MAX_LENGTH'] = 520
hyper_params['PADDING_TYPE'] = "post"
hyper_params['TRUNC_TYPE'] = "post"

In [ ]:
# perform padding on the training and testing dataset

training_padded = pad_sequences(train_sequences, padding=hyper_params['PADDING_TYPE'], 
                                maxlen=hyper_params['MAX_LENGTH'], truncating=hyper_params['TRUNC_TYPE'])
testing_padded = pad_sequences(test_sequences, padding=hyper_params['PADDING_TYPE'], 
                               maxlen=hyper_params['MAX_LENGTH'], truncating=hyper_params['TRUNC_TYPE'])


print(len(testing_padded))

10000


In [ ]:
# Change categorical data to numerical

train_labels = []
test_labels = []

for item in y_train:
    if item == 'positive':
        train_labels.append(1)
    else:
        train_labels.append(0)
        
for item in y_test:
    if item == 'positive':
        test_labels.append(1)
    else:
        test_labels.append(0)

train_labels = np.asarray(train_labels).astype('float32')
test_labels = np.asarray(test_labels).astype('float32')

#2 Conv Layer CNN


We will construct a 2-layer CNN to observe its behaviour. 

In [ ]:
# Create a "basic" 2 layer CNN 

model_2CNN_basic = keras.Sequential([
    keras.layers.Embedding(len(word_index)+1, 20, input_length=520),
    
    keras.layers.Conv1D(40, 3, activation='relu'),
    keras.layers.MaxPooling1D(padding='same'),
    keras.layers.Dropout(0.3),
    
    keras.layers.Conv1D(20, 3, activation='relu'),
    keras.layers.MaxPooling1D(padding='same'),
    keras.layers.Dropout(0.3),
    
    keras.layers.Flatten(),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

model_2CNN_basic.compile(loss='binary_crossentropy', 
                           optimizer=keras.optimizers.Adam(learning_rate=hyper_params['LR']), 
                           metrics=['accuracy'])
model_2CNN_basic.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 520, 20)           2244520   
                                                                 
 conv1d (Conv1D)             (None, 518, 40)           2440      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 259, 40)          0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 259, 40)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 257, 20)           2420      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 129, 20)          0         
 1D)                                                    

In [ ]:
# Train the basic 2-layer CNN

history_2CNN_basic = model_2CNN_basic.fit(training_padded, train_labels, batch_size = hyper_params['BS'],
                        epochs=hyper_params['EPOCHS'], validation_data=(testing_padded, test_labels))

Epoch 1/150
79/79 [==============================] - 48s 583ms/step - loss: 0.6288 - accuracy: 0.6290 - val_loss: 0.4700 - val_accuracy: 0.7852
Epoch 2/150
79/79 [==============================] - 46s 582ms/step - loss: 0.4063 - accuracy: 0.8239 - val_loss: 0.3892 - val_accuracy: 0.8333
Epoch 3/150
79/79 [==============================] - 42s 538ms/step - loss: 0.2987 - accuracy: 0.8820 - val_loss: 0.3580 - val_accuracy: 0.8502
Epoch 4/150
79/79 [==============================] - 44s 553ms/step - loss: 0.2184 - accuracy: 0.9189 - val_loss: 0.3218 - val_accuracy: 0.8730
Epoch 5/150
79/79 [==============================] - 44s 558ms/step - loss: 0.1490 - accuracy: 0.9488 - val_loss: 0.3194 - val_accuracy: 0.8886
Epoch 6/150
79/79 [==============================] - 43s 543ms/step - loss: 0.1091 - accuracy: 0.9622 - val_loss: 0.4686 - val_accuracy: 0.8769
Epoch 7/150
79/79 [==============================] - 47s 600ms/step - loss: 0.0844 - accuracy: 0.9718 - val_loss: 0.3993 - val_accuracy:

In [ ]:
# Save loss figure

fig = go.Figure()
fig.add_trace(go.Scatter(x=history_2CNN_basic.epoch, y=history_2CNN_basic.history['loss'], name='Training Loss'))
fig.add_trace(go.Scatter(x=history_2CNN_basic.epoch, y=history_2CNN_basic.history['val_loss'], name='Validation Loss'))

fig.show()
fig.write_html('basic_2_CNN_loss.html')


In [ ]:
# Save accuracy figure

fig = go.Figure()
fig.add_trace(go.Scatter(x=history_2CNN_basic.epoch, y=history_2CNN_basic.history['accuracy'], name='Training Accuracy'))
fig.add_trace(go.Scatter(x=history_2CNN_basic.epoch, y=history_2CNN_basic.history['val_accuracy'], name='Validation Accuracy'))

fig.show()
fig.write_html('basic_2_CNN_acc.html')
#fig.write_image('basic_2_CNN_acc.jpeg')

In [ ]:
# Save history as csv and json

# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history_2CNN_basic.history) 

# save to csv:
hist_csv_file = 'history_2CNN_basic.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

#3 Conv Layer CNN


We will construct a 3-layer CNN to observe its behaviour. 

In [ ]:
# Create a "basic" 3-layer CNN

model_3CNN_basic = keras.Sequential([
    keras.layers.Embedding(len(word_index)+1, 20, input_length=520),
    keras.layers.Conv1D(100, 3, activation='relu'),
    keras.layers.MaxPooling1D(padding='same'),
    keras.layers.Dropout(0.2),
    keras.layers.Conv1D(70, 3, activation='relu'),
    keras.layers.MaxPooling1D(padding='same'),
    keras.layers.Dropout(0.3),
    keras.layers.Conv1D(40, 3, activation='relu'),
    keras.layers.MaxPooling1D(padding='same'),
    keras.layers.Dropout(0.5),
    keras.layers.Flatten(),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

model_3CNN_basic.compile(loss='binary_crossentropy', 
                           optimizer=keras.optimizers.Adam(learning_rate=hyper_params['LR']), 
                           metrics=['accuracy'])
model_3CNN_basic.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 520, 20)           2244520   
                                                                 
 conv1d (Conv1D)             (None, 518, 100)          6100      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 259, 100)         0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 259, 100)          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 257, 70)           21070     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 129, 70)          0         
 1D)                                                    

In [ ]:
# Train the basic 3-layer CNN

history_3CNN_basic = model_3CNN_basic.fit(training_padded, train_labels, batch_size = hyper_params['BS'],
                        epochs=hyper_params['EPOCHS'], validation_data=(testing_padded, test_labels))

Epoch 1/150
79/79 [==============================] - 76s 950ms/step - loss: 0.6958 - accuracy: 0.5015 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/150
79/79 [==============================] - 73s 929ms/step - loss: 0.6932 - accuracy: 0.5020 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/150
79/79 [==============================] - 73s 929ms/step - loss: 0.6933 - accuracy: 0.5012 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/150
79/79 [==============================] - 73s 928ms/step - loss: 0.6932 - accuracy: 0.4993 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/150
79/79 [==============================] - 73s 924ms/step - loss: 0.6932 - accuracy: 0.4994 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 6/150
79/79 [==============================] - 73s 930ms/step - loss: 0.6932 - accuracy: 0.4987 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 7/150
79/79 [==============================] - 73s 918ms/step - loss: 0.6932 - accuracy: 0.5017 - val_loss: 0.6932 - val_accuracy:

In [ ]:
# Save loss figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=history_3CNN_basic.epoch, y=history_3CNN_basic.history['loss'], name='Training Loss'))
fig.add_trace(go.Scatter(x=history_3CNN_basic.epoch, y=history_3CNN_basic.history['val_loss'], name='Validation Loss'))

fig.show()
fig.write_html('basic_3_CNN_loss.html')
#fig.write_image('basic_3_CNN_loss.jpeg')

In [ ]:
# Save accuracy figure

fig = go.Figure()
fig.add_trace(go.Scatter(x=history_3CNN_basic.epoch, y=history_3CNN_basic.history['accuracy'], name='Training Accuracy'))
fig.add_trace(go.Scatter(x=history_3CNN_basic.epoch, y=history_3CNN_basic.history['val_accuracy'], name='Validation Accuracy'))

fig.show()
fig.write_html('basic_3_CNN_acc.html')
#fig.write_image('basic_3_CNN_acc.jpeg')

In [ ]:
# Save history as csv and json

# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history_3CNN_basic.history) 

# save to csv:
hist_csv_file = 'history_3CNN_basic.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

#4 Conv Layer CNN


We will construct a 4-layer CNN to observe its behaviour.

In [ ]:
# Create a "basic" 4-layer CNN

model_4CNN_basic = keras.Sequential([
    keras.layers.Embedding(len(word_index)+1, 20, input_length=520),
    keras.layers.Conv1D(100, 3, activation='relu'),
    keras.layers.MaxPooling1D(padding='same'),
    keras.layers.Dropout(0.2),
    keras.layers.Conv1D(70, 3, activation='relu'),
    keras.layers.MaxPooling1D(padding='same'),
    keras.layers.Dropout(0.3),
    keras.layers.Conv1D(40, 3, activation='relu'),
    keras.layers.MaxPooling1D(padding='same'),
    keras.layers.Dropout(0.4),
    keras.layers.Conv1D(20, 3, activation='relu'),
    keras.layers.MaxPooling1D(padding='same'),
    keras.layers.Dropout(0.5),
    keras.layers.Flatten(),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

model_4CNN_basic.compile(loss='binary_crossentropy', 
                           optimizer=keras.optimizers.Adam(learning_rate=hyper_params['LR']), 
                           metrics=['accuracy'])
model_4CNN_basic.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 520, 20)           2244520   
                                                                 
 conv1d (Conv1D)             (None, 518, 100)          6100      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 259, 100)         0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 259, 100)          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 257, 70)           21070     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 129, 70)          0         
 1D)                                                    

In [ ]:
# Train the basic 4-layer CNN

history_4CNN_basic = model_4CNN_basic.fit(training_padded, train_labels, batch_size = hyper_params['BS'],
                        epochs=hyper_params['EPOCHS'], validation_data=(testing_padded, test_labels))

Epoch 1/150
79/79 [==============================] - 76s 954ms/step - loss: 0.6933 - accuracy: 0.5052 - val_loss: 0.6806 - val_accuracy: 0.6105
Epoch 2/150
79/79 [==============================] - 75s 944ms/step - loss: 0.4743 - accuracy: 0.7850 - val_loss: 0.3994 - val_accuracy: 0.8043
Epoch 3/150
79/79 [==============================] - 75s 949ms/step - loss: 0.2790 - accuracy: 0.8920 - val_loss: 0.3469 - val_accuracy: 0.8479
Epoch 4/150
79/79 [==============================] - 74s 932ms/step - loss: 0.1763 - accuracy: 0.9357 - val_loss: 0.3190 - val_accuracy: 0.8611
Epoch 5/150
79/79 [==============================] - 74s 935ms/step - loss: 0.1245 - accuracy: 0.9564 - val_loss: 0.2695 - val_accuracy: 0.8960
Epoch 6/150
79/79 [==============================] - 74s 934ms/step - loss: 0.0848 - accuracy: 0.9703 - val_loss: 0.3514 - val_accuracy: 0.8824
Epoch 7/150
79/79 [==============================] - 74s 932ms/step - loss: 0.0615 - accuracy: 0.9781 - val_loss: 0.6434 - val_accuracy:

In [ ]:
# Save loss figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=history_4CNN_basic.epoch, y=history_4CNN_basic.history['loss'], name='Training Loss'))
fig.add_trace(go.Scatter(x=history_4CNN_basic.epoch, y=history_4CNN_basic.history['val_loss'], name='Validation Loss'))

fig.show()
fig.write_html('basic_4_CNN_loss.html')
#fig.write_image('basic_4_CNN_loss.jpeg')

In [ ]:
# Save accuracy figure

fig = go.Figure()
fig.add_trace(go.Scatter(x=history_4CNN_basic.epoch, y=history_4CNN_basic.history['accuracy'], name='Training Accuracy'))
fig.add_trace(go.Scatter(x=history_4CNN_basic.epoch, y=history_4CNN_basic.history['val_accuracy'], name='Validation Accuracy'))

fig.show()
fig.write_html('basic_4_CNN_acc.html')
#fig.write_image('basic_4_CNN_acc.jpeg')

In [ ]:
# Save history as csv and json

# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history_4CNN_basic.history) 

# save to csv:
hist_csv_file = 'history_4CNN_basic.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)